In [ ]:
from athletes import *
from secrets import *
from stravalib.client import Client     # library used for access
from stravaio import strava_oauth2      # just for authentication to get token
from stravaio import StravaIO
from pprint import pprint
import numpy as np
import pandas as pd
import requests

In [ ]:
from secrets import *
from athletes import *
from clubs import *

In [ ]:
# get authentification
auth = strava_oauth2(client_id=STRAVA_CLIENT_ID, client_secret=STRAVA_CLIENT_SECRET)
access_token = auth['access_token']

In [ ]:
# urls
athlete_url = "https://www.strava.com/api/v3/athlete"
activities_url = "https://www.strava.com/api/v3/athlete/activities"
clubs_url = "https://www.strava.com/api/v3/clubs"

# request header
header = {'Authorization':'Bearer ' + access_token}
param = {'per_page':200, 'page':1}


In [ ]:
# get athlete
athlete = requests.get(athlete_url, headers=header, params=param).json()

In [ ]:
# get clubs the athlete is member of
clubs = requests.get(clubs_url, headers=header, params=param).json()

In [ ]:
request_url = clubs_url + "/" + CLUB_CCB + "/activities"
ccb_activities = requests.get(request_url, headers=header, params = param).json()

In [ ]:
# get athletes from list of activities
athletes = list()
athlete_names = list()
for activity in ccb_activities:
    if activity['athlete']['firstname'] not in athlete_names:
        athletes.append(activity['athlete'])
        athlete_names.append(activity['athlete']['firstname'])

athletes_df = pd.DataFrame.from_records(athletes)
print(athletes_df)

In [ ]:
# prepare empty lists for distances
distance = list()
for athlete in athlete_names:
    distance.append(list())

for activity in ccb_activities:
    current_athlete = athlete_names.index(activity['athlete']['firstname'])
    distance[current_athlete].append(activity['distance'])

# determine total distance for each athlete
total_distance = list()
print('Total distance of each club member:')
for idx, athlete in enumerate(athlete_names):
    distance_athlete = sum(distance[idx])
    total_distance.append(distance_athlete)
    print('\t' + athlete + ': ', "{:.2f}".format(distance_athlete/1000), 'km')

# determine highest distance and the athlete with it
max_distance = max(total_distance)
max_athlete = athlete_names[total_distance.index(max_distance)]
print('=> The athlete with the most ridden km so far is ', max_athlete, ', with a total of ', "{:.2f}".format(max_distance/1000), 'km!')